In [3]:
import gc
import cv2
import dlib
from PIL import Image
import imutils
import numpy as np
import tensorflow as tf
from sklearn.decomposition import PCA
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

In [4]:
# Displayinh the number of GPUs available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(False)

Num GPUs Available:  1


In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

In [6]:
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

2023-02-11 12:35:39.660880: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-11 12:35:40.652467: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20807 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0


In [7]:
# load a test image, resize it, and convert it to grayscale
def load_image(path, show=False, gray_show=False):
    test_image = cv2.imread(path)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)

    test_image = imutils.resize(test_image, width=500)
    gray = cv2.cvtColor(test_image, cv2.COLOR_RGB2GRAY)

    if show:
        plt.imshow(test_image)
        plt.show()
        if gray_show:
            plt.imshow(gray, cmap='gray')
            plt.show()
    return test_image, gray

def detect_faces(face_detector, gray, img, scale_factor, frame_num, show_multi_faces = False):
    detections = haar_cascade.detectMultiScale(gray, scaleFactor=scale_factor, minNeighbors=5, minSize=(150, 150), flags=cv2.CASCADE_SCALE_IMAGE)
    #Checking if there are more than 1 detected faces
    if len(detections) > 1:
        print('multiple faces', frame_num)
        if detections[0][2] > detections[1][2]:
            face = detections[1]
            other = detections[0]
        else:
            face = detections[0]
            other = detections[1]
        if show_multi_faces:
            print('largest and chosen face')
            fX, fY, fW, fH = face[0],face[1], face[2], face[3]
            plt.imshow(gray[fY:fY + fH, fX:fX + fW], cmap='gray')
            plt.show()
            print('not chosen face')
            fX, fY, fW, fH = other[0], other[1], other[2], other[3]
            plt.imshow(gray[fY:fY + fH, fX:fX + fW], cmap='gray')
            plt.show()
    elif len(detections) == 1:
        face = detections[0]
    # if no face detected
    elif len(detections) == 0:
        print('No face detected', frame_num)
        return False
    fX, fY, fW, fH = face[0], face[1], face[2], face[3]
    face_img = Image.fromarray(img[fY:fY + fH, fX:fX + fW])
    face_img = face_img.resize((224, 224))
    #face_img.show()
    return face_img

In [8]:
# Specify subject and condition
subject = 'P11401'
condition = 'sleepy'

In [10]:
# haar Cascade path
cascade_path = '../models/haarcascade_frontalface_default.xml'
# create haar cascade
haar_cascade = cv2.CascadeClassifier(cascade_path)
face_detector = 'haar'

frame_start = 47000
frame_end = 210000
frames = range(frame_start, frame_end + 1)

scale_factor = 1.04

folder_path = '../frames/' + subject + '/' + condition + '/frame'

In [ ]:
def get_embeddings(frames, folder_path, face_detector, model, scale_factor, show=False, show_multi_faces=False):
    no_face_count = 0
    embeddings = []
    for frame in frames:
        if frame % 1000 == 0:
            print(frame)
            gc.collect()
        if frame == 100000:
            np.save('embeddings_sub' + subject[:-2] + '_' + condition + '_temp.npy', embeddings)
        path = folder_path + str(frame) + '.jpg'
        test_img, gray = load_image(path, show=False)
        face = detect_faces(face_detector, gray, test_img, scale_factor, frame, show_multi_faces=False)

        if not face:
            print("No face detected", frame)
            no_face_count += 1
            continue

        face_array = np.asarray(face)
        pixels = face_array.astype('float32')
        samples = np.expand_dims(pixels, axis=0)
        samples = preprocess_input(samples, version=2)
        embedding = model.predict(samples, verbose=0)
        embeddings.append(embedding[0])
    print(no_face_count)
    return embeddings

embeddings = get_embeddings(frames, folder_path, face_detector, model, scale_factor)
np.save('embeddings_sub' + subject[:-2] + '_' + condition + '.npy', embeddings)

49000
No face detected 49034
No face detected 49034
No face detected 49039
No face detected 49039
No face detected 49042
No face detected 49042
No face detected 49044
No face detected 49044
No face detected 49045
No face detected 49045
No face detected 49046
No face detected 49046
No face detected 49047
No face detected 49047
No face detected 49048
No face detected 49048
No face detected 49049
No face detected 49049
No face detected 49050
No face detected 49050
No face detected 49051
No face detected 49051
No face detected 49052
No face detected 49052
No face detected 49053
No face detected 49053
No face detected 49054
No face detected 49054
No face detected 49056
No face detected 49056
No face detected 49057
No face detected 49057
No face detected 49058
No face detected 49058
No face detected 49059
No face detected 49059
No face detected 49060
No face detected 49060
No face detected 49061
No face detected 49061
No face detected 49062
No face detected 49062
No face detected 49063
No fa

In [ ]:
print(len(embeddings[:-17000]))
np.save('embeddings_sub10_sleepy.npy', embeddings[:-17000])

In [ ]:
# Extracts average embedding in a segment from given embeddigs
def embeddings_segment(embeddings, video_len, segment_len):
    # The amount of frames at the end that are not taken into account
    rest = video_len % segment_len
    num_frames = video_len - rest
    avg_embeddings = []
    acum_embeddings = np.zeros(2048)
    # a blink is counted to a segment,when the blink starts in that segment
    for frame in range(num_frames + 1):
        if frame % 1000 == 0:
            print(frame)
        acum_embeddings = acum_embeddings + np.array(embeddings[frame])
        # only happens at the end of a segment
        if frame % segment_len == 0 and frame != 0:
            avg_embeddings.append(acum_embeddings / segment_len)
            acum_embeddings = np.zeros(2048)
            #print('New segment: ', frame)
    return avg_embeddings

avg_embeddings = embeddings_segment(embeddings, len(embeddings), 100)

In [ ]:
pca = PCA(n_components=2)
pri_comps = pca.fit_transform(avg_embeddings)

print(pri_comps)